<a href="https://colab.research.google.com/github/2pterons/multicampus/blob/main/GPT2_Idiom_effector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# cuda version check
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
#%cd '/content/drive/MyDrive/mulcam_project/BERT_idiom'

In [ ]:
!wget -q -nc http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf /content/aclImdb_v1.tar.gz

In [ ]:
# Install transformers library.
!pip3 install -q git+https://github.com/huggingface/transformers.git
# Install helper functions.
!pip3 install -q git+https://github.com/gmihaila/ml_things.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 5.2 MB/s 
     |████████████████████████████████| 636 kB 41.0 MB/s 
     |████████████████████████████████| 3.3 MB 35.0 MB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
     |████████████████████████████████| 64 kB 2.0 MB/s 
     |████████████████████████████████| 10.3 MB 10.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
#!pip install transformers

In [ ]:
'''import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
import pickle'''

#########################################################################################
import io
import os
import torch
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from ml_things import plot_dict, plot_confusion_matrix, fix_text
from sklearn.metrics import classification_report, accuracy_score
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW, 
                          get_linear_schedule_with_warmup,
                          GPT2ForSequenceClassification)
# 시드 설정
set_seed(123)

# epochs 설정(2~4권장)
epochs = 4

# batch_size 설정
batch_size = 32

# padding or truncate
max_length = 60

# gpu 설정(오류시 cpu 사용)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 설정(gpt2 사용)
model_name_or_path = 'gpt2'

# 분류 설정
labels_ids = {'neg':0, 'pos':1}

# 사용될 라벨 갯수 설정, 분류 헤드 크기 설정
n_labels = len(labels_ids)

In [ ]:
# fix_text = 유니코드 문제 해결
# init() 데이터 집합에서 읽고 텍스트와 레이블을 숫자로 변환
# len() 읽은 예제 수를 반환해야하는 위치를 나타냄, Movie Rviews Dataset() 호출시 사용
# getitem() 항상 입력으로 사용되는데, 이 int 값은 예제의 어떤 예제를 데이터 집합에서 반환할 인트 값을 나타낸다?
# 값 3이 통과된면 위치 3의 데이터셋 예제를 반환.

In [ ]:
"""
Gpt2ClassificationCollator
이 클래스를 사용하여 데이터 수집기를 만든다.
DataLoader에서 모델에 공급되는 데이터 배시를 만드는데 사용된다.
각 시퀀스의 토크나이저와 레이블 인코더를 사용하여 텍스트와 레이블 번호로 변환
"""

'\nGpt2ClassificationCollator\n이 클래스를 사용하여 데이터 수집기를 만든다.\nDataLoader에서 모델에 공급되는 데이터 배시를 만드는데 사용된다.\n각 시퀀스의 토크나이저와 레이블 인코더를 사용하여 텍스트와 레이블 번호로 변환\n'

In [ ]:
#%cd /content/drive/MyDrive/Colab Notebooks/multicampus/Idiom Classifier/GPT2/
import moviereviewsdataset
import gpt2classificationcollator

def train(dataloader, optimizer_, scheduler_, device_):
  r"""
  Train pytorch model on a single pass through the data loader.

  It will use the global variable `model` which is the transformer model 
  loaded on `_device` that we want to train on.

  This function is built with reusability in mind: it can be used as is as long
    as the `dataloader` outputs a batch in dictionary format that can be passed 
    straight into the model - `model(**batch)`.

  Arguments:

      dataloader (:obj:`torch.utils.data.dataloader.DataLoader`):
          Parsed data into batches of tensors.

      optimizer_ (:obj:`transformers.optimization.AdamW`):
          Optimizer used for training.

      scheduler_ (:obj:`torch.optim.lr_scheduler.LambdaLR`):
          PyTorch scheduler.

      device_ (:obj:`torch.device`):
          Device used to load tensors before feeding to model.

  Returns:

      :obj:`List[List[int], List[int], float]`: List of [True Labels, Predicted
        Labels, Train Average Loss].
  """

  # Use global variable for model. 모델에 전역 변수를 사용합니다.
  global model

  # Tracking variables. 추적 변수
  predictions_labels = []
  true_labels = []
  # Total loss for this epoch. 이 에포크의 총 손실입니다.
  total_loss = 0

  # Put the model into training mode. 모델을 교육 모드로 전환합니다.
  model.train()

  # For each batch of training data... 각 교육 데이터 배치에 대해 다음을 수행합니다.
  for batch in tqdm(dataloader, total=len(dataloader)):

    # Add original labels - use later for evaluation. 원본 레이블 추가 - 나중에 평가에 사용합니다.
    true_labels += batch['labels'].numpy().flatten().tolist()
    
    # move batch to device 배치를 장치로 이동합니다.
    batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}
    
    # Always clear any previously calculated gradients before performing a
    # backward pass. 후진 패스를 수행하기 전에 항상 이전에 계산된 구배를 지웁니다.
    model.zero_grad()

    # Perform a forward pass (evaluate the model on this training batch). 전진 통과를 수행합니다(이 교육 배치에서 모델 평가).
    # This will return the loss (rather than the model output) because we
    # have provided the `labels`.
    # The documentation for this a bert model function is here: 
    # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
    """ Perform a forward pass (evaluate the model on this training batch).
    # This will return the loss (rather than the model output) because we have provided the `labels`.
    # The documentation for this a bert model function is here: 
    # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
    """
    
    outputs = model(**batch)

    # The call to `model` always returns a tuple, so we need to pull the 
    # loss value out of the tuple along with the logits. We will use logits
    # later to calculate training accuracy.
    """
    # '모델'에 대한 호출은 항상 튜플을 반환하므로, 우리는 튜플을 당겨야 합니다.
    # 로짓과 함께 튜플에서 손실 값입니다. 나중에 로짓을 사용하여 교육 정확도를 계산합니다.
    """
    loss, logits = outputs[:2]

    # Accumulate the training loss over all of the batches so that we can
    # calculate the average loss at the end. `loss` is a Tensor containing a
    # single value; the `.item()` function just returns the Python value 
    # from the tensor.
    """
    # 모든 배치에 걸쳐 교육 손실을 누적하여 마지막에 평균 손실을 계산할 수 있도록 합니다. 
    'loss'는 단일 값을 포함하는 텐서이며, '.item()' 함수는 텐서로부터 파이썬 값을 반환하기만 합니다.
    """
    total_loss += loss.item()

    # Perform a backward pass to calculate the gradients.
    # 뒤로 통과를 수행하여 구배를 계산합니다.
    loss.backward()

    # Clip the norm of the gradients to 1.0.
    # This is to help prevent the "exploding gradients" problem.
    """
    구배의 규범을 1.0에 자릅니다.
    이는 "분할 그라데이션" 문제를 방지하기 위한 것입니다.
    """
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # Update parameters and take a step using the computed gradient.
    # The optimizer dictates the "update rule"--how the parameters are
    # modified based on their gradients, the learning rate, etc.
    """
    # 매개변수를 업데이트하고 계산된 그라데이션으로 단계를 수행합니다.
    # Optimizer는 "업데이트 규칙"을 지정합니다. 즉, 파라미터의 기울기, 
      학습 속도 등에 따라 파라미터가 수정되는 방식을 지정합니다.
    """
    optimizer.step()

    # Update the learning rate. 학습 속도를 업데이트합니다.
    scheduler.step()

    # Move logits and labels to CPU 로그와 레이블을 CPU로 이동합니다.
    logits = logits.detach().cpu().numpy()

    # Convert these logits to list of predicted labels values.
    # 이러한 로짓을 예측 레이블 값 목록으로 변환합니다.
    predictions_labels += logits.argmax(axis=-1).flatten().tolist()

  # Calculate the average loss over the training data.
  # 교육 데이터에 대한 평균 손실을 계산합니다.
  avg_epoch_loss = total_loss / len(dataloader)
  
  # Return all true labels and prediction for future evaluations.
  # 향후 평가를 위해 모든 참 레이블과 예측을 반환합니다.
  return true_labels, predictions_labels, avg_epoch_loss



def validation(dataloader, device_):
  r"""Validation function to evaluate model performance on a 
  separate set of data.

  This function will return the true and predicted labels so we can use later
  to evaluate the model's performance.

  This function is built with reusability in mind: it can be used as is as long
    as the `dataloader` outputs a batch in dictionary format that can be passed 
    straight into the model - `model(**batch)`.

  Arguments:

    dataloader (:obj:`torch.utils.data.dataloader.DataLoader`):
          Parsed data into batches of tensors.

    device_ (:obj:`torch.device`):
          Device used to load tensors before feeding to model.

  Returns:
    
    :obj:`List[List[int], List[int], float]`: List of [True Labels, Predicted
        Labels, Train Average Loss]
  """

  # Use global variable for model.
  # 모형에 전역 변수를 사용합니다.
  global model

  # Tracking variables 추적 변수입니다.
  predictions_labels = []
  true_labels = []
  #total loss for this epoch. 이 에포크의 총 손실입니다.
  total_loss = 0

  # Put the model in evaluation mode--the dropout layers behave differently
  # during evaluation.
  # 모델을 평가 모드로 전환합니다. 평가 중에 드롭아웃 계층이 다르게 동작합니다.
  model.eval()

  # 한 시대에 대한 데이터를 평가합니다.
  for batch in tqdm(dataloader, total=len(dataloader)):

    # add original labels 원본 레이블을 추가합니다.
    true_labels += batch['labels'].numpy().flatten().tolist()

    # move batch to device 배치를 장치로 이동합니다.
    batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}

    # Telling the model not to compute or store gradients, saving memory and
    # speeding up validation
    # 모델에 그라데이션 계산이나 저장을 하지 않도록 하고 메모리를 절약하며 검증 속도를 높입니다.
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have
        # not provided labels.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        """
        정방향 통과, 로짓 예측을 계산합니다.
      # 라벨이 제공되지 않았으므로 손실보다는 로짓이 반환됩니다.
      # token_type_ids는 2-token 태스크에서 문장 1과 2를 구분하는 "세그먼트 ID"와 동일합니다.
      # 이 '모델' 기능에 대한 설명서는 다음과 같습니다.
      # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers을 참조하십시오.BertForSequenceClassification입니다.
        """
        outputs = model(**batch)

        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple along with the logits. We will use logits
        # later to to calculate training accuracy.
        """
        '모델' 호출은 항상 튜플을 반환하므로 로짓과 함께 손실 값을 튜플에서 빼내야 합니다. 
        나중에 로짓을 사용하여 교육 정확도를 계산할 것입니다.
        """
        loss, logits = outputs[:2]
        
        # Move logits and labels to CPU 로그와 레이블을 CPU로 이동합니다.
        logits = logits.detach().cpu().numpy()

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        """
        모든 배치에 걸쳐 교육 손실을 누적하여 마지막에 평균 손실을 계산할 수 있도록 합니다. 
        'loss'는 단일 값을 포함하는 텐서이며, '.item()' 함수는 텐서로부터 파이썬 값을 반환하기만 합니다.
        """
        total_loss += loss.item()
        
        # get predicitons to list 예측치를 열거합니다.
        predict_content = logits.argmax(axis=-1).flatten().tolist()

        # update list 목록을 업데이트합니다.
        predictions_labels += predict_content

  # Calculate the average loss over the training data. 교육 데이터에 대한 평균 손실을 계산합니다.
  avg_epoch_loss = total_loss / len(dataloader)

  # Return all true labels and prediciton for future evaluations.
  # 향후 평가를 위해 모든 참 라벨과 예측을 반환합니다.
  return true_labels, predictions_labels, avg_epoch_loss

NameError: ignored

In [ ]:
# Get model configuration. 모델 구성을 가져옵니다.
print('Loading configuraiton...')
model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path=model_name_or_path, num_labels=n_labels)

# Get model's tokenizer. 모델의 토큰라이저를 가져옵니다.
print('Loading tokenizer...')
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path=model_name_or_path)
# default to left padding 기본값은 왼쪽 패딩입니다.
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token = 50256 / PAD 토큰 = EOS 토큰 = 50256을 정의합니다.
tokenizer.pad_token = tokenizer.eos_token


# Get the actual model. 실제 모델을 가져옵니다.
print('Loading model...')
model = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name_or_path, config=model_config)

# resize model embedding to match new tokenizer 모델 크기 조정: 새 토큰라이저와 일치하도록 내장되어 있습니다.
model.resize_token_embeddings(len(tokenizer))

# fix model padding token id 모델 패딩 토큰 ID를 수정합니다.
model.config.pad_token_id = model.config.eos_token_id

# Load model to defined device. Load model to defined device.
# 모델을 정의된 장치에 로드합니다. 모델을 정의된 장치에 로드합니다.
model.to(device)
print('Model loaded to `%s`'%device)

Loading configuraiton...


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Loading tokenizer...


Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loading model...


Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded to `cuda`


In [ ]:
# Create data collator to encode text and labels into numbers.
# 텍스트와 레이블을 숫자로 인코딩할 데이터 수집기를 만듭니다.
gpt2_classificaiton_collator = Gpt2ClassificationCollator(use_tokenizer=tokenizer, 
                                                          labels_encoder=labels_ids, 
                                                          max_sequence_len=max_length)


print('Dealing with Train...')
# Create pytorch dataset.
# Pytorch 데이터 집합을 만듭니다.
import sys
sys.path.join(0, '/content/drive/MyDrive/Colab Notebooks/multicampus/Idiom Classifier/data/' )
train_dataset = MovieReviewsDataset(path='/content/drive/MyDrive/Colab Notebooks/multicampus/Idiom Classifier/data/', 
                               use_tokenizer=tokenizer)
print('Created `train_dataset` with %d examples!'%len(train_dataset))

# Move pytorch dataset into dataloader. 파이토치 데이터 집합을 데이터 로더로 이동합니다.
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=gpt2_classificaiton_collator)
print('Created `train_dataloader` with %d batches!'%len(train_dataloader))

print()

print('Dealing with Validation...')
# Create pytorch dataset. Pytorch 데이터 집합을 만듭니다.
valid_dataset =  MovieReviewsDataset(path='/content/drive/MyDrive/Colab Notebooks/multicampus/Idiom Classifier/data/', 
                               use_tokenizer=tokenizer)
print('Created `valid_dataset` with %d examples!'%len(valid_dataset))

# Move pytorch dataset into dataloader. 파이토치 데이터 집합을 데이터 로더로 이동합니다
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=gpt2_classificaiton_collator)
print('Created `eval_dataloader` with %d batches!'%len(valid_dataloader))

Dealing with Train...


FileNotFoundError: ignored

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
"""
참고: Adam W는 포옹 페이스 라이브러리의 클래스입니다(피토치 반대). 'W'는 'Weight Decay fix'의 약자라고 생각합니다.
"""
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # default is 1e-8.
                  )

# Total number of training steps is number of batches * number of epochs.
# `train_dataloader` contains batched data so `len(train_dataloader)` gives 
# us the number of batches.
"""
총 교육 단계 수는 배치 수 * epoch 수입니다.
'train_dataloader'에는 일괄 데이터가 포함되어 있으므로 'len(train_dataloader)'은 다음을 제공합니다.
우리에게 묶음의 수를 알려줍니다.
"""
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# 학습 속도 스케줄러를 만듭니다.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

# Store the average loss after each epoch so we can plot them.
# 각 시대 이후의 평균 손실을 저장하여 표시할 수 있습니다.
all_loss = {'train_loss':[], 'val_loss':[]}
all_acc = {'train_acc':[], 'val_acc':[]}

# Loop through each epoch.
# 각 시대를 루프합니다.
print('Epoch')
for epoch in tqdm(range(epochs)):
  print()
  print('Training on batches...')
  # Perform one full pass over the training set.]
  # 교육 세트 위에 풀 패스를 한 번 수행합니다.
  train_labels, train_predict, train_loss = train(train_dataloader, optimizer, scheduler, device)
  train_acc = accuracy_score(train_labels, train_predict)

  # Get prediction form model on validation data. 
  # 검증 데이터에 대한 예측 양식 모형을 가져옵니다.
  print('Validation on batches...')
  valid_labels, valid_predict, val_loss = validation(valid_dataloader, device)
  val_acc = accuracy_score(valid_labels, valid_predict)

  # Print loss and accuracy values to see how training evolves.
  # 손실 및 정확도 값을 인쇄하여 교육이 어떻게 발전하는지 확인할 수 있습니다.
  print("  train_loss: %.5f - val_loss: %.5f - train_acc: %.5f - valid_acc: %.5f"%(train_loss, val_loss, train_acc, val_acc))
  print()

  # Store the loss value for plotting the learning curve.
  # 학습 곡선을 표시하기 위한 손실 값을 저장합니다.
  all_loss['train_loss'].append(train_loss)
  all_loss['val_loss'].append(val_loss)
  all_acc['train_acc'].append(train_acc)
  all_acc['val_acc'].append(val_acc)

# Plot loss curves.
# 손실 곡선을 표시합니다.
plot_dict(all_loss, use_xlabel='Epochs', use_ylabel='Value', use_linestyles=['-', '--'])

# Plot accuracy curves.
# 정확도 곡선을 표시합니다.
plot_dict(all_acc, use_xlabel='Epochs', use_ylabel='Value', use_linestyles=['-', '--'])

In [ ]:
# Get prediction form model on validation data. This is where you should use
# your test data.
# 검증 데이터에 대한 예측 양식 모형을 가져옵니다. 여기서 테스트 데이터를 사용해야 합니다.
true_labels, predictions_labels, avg_epoch_loss = validation(valid_dataloader, device)

# Create the evaluation report.
# 평가 보고서를 생성합니다.
evaluation_report = classification_report(true_labels, predictions_labels, labels=list(labels_ids.values()), target_names=list(labels_ids.keys()))
# Show the evaluation report.
# 평가 보고서를 표시합니다.
print(evaluation_report)

# Plot confusion matrix.
# 혼동 행렬을 표시합니다.
plot_confusion_matrix(y_true=true_labels, y_pred=predictions_labels, 
                      classes=list(labels_ids.keys()), normalize=True, 
                      magnify=0.1,
                      );

In [ ]:
"""
%cd /content/drive/MyDrive/Colab Notebooks/multicampus/bert_naver_movie/data/
import pickle
with open('TroFi_dataset.pkl', 'rb') as f:
  corpus = pickle.load(f)
"""